This notebook is to process level 1 AIA data to level 1.5 data and save it

In [6]:
import warnings
warnings.filterwarnings('ignore')

import os
import glob
import sunpy
from aiapy.calibrate import register, update_pointing

data_dir = '/home/mnedal/data'

In [37]:
def split_datetime(start=None, end=None):
    
    START_DATE, START_TIME = start.split('T')
    END_DATE, END_TIME = end.split('T')

    START_YEAR, START_MONTH, START_DAY = START_DATE.split('-')
    END_YEAR, END_MONTH, END_DAY = END_DATE.split('-')

    START_HOUR, START_MINUTE, START_SECOND = START_TIME.split(':')
    END_HOUR, END_MINUTE, END_SECOND = END_TIME.split(':')

    datetime_dict = {
        'start_year': START_YEAR,
        'start_month': START_MONTH,
        'start_day': START_DAY,
        'start_hour': START_HOUR,
        'start_minute': START_MINUTE,
        'start_second': START_SECOND,
        
        'end_year': END_YEAR,
        'end_month': END_MONTH,
        'end_day': END_DAY,
        'end_hour': END_HOUR,
        'end_minute': END_MINUTE,
        'end_second': END_SECOND
    }
    return datetime_dict



def load_aia(start=None, end=None, promote=False, channel=193):
    
    dt_dict = split_datetime(start=start, end=end)
    data_path = f'{data_dir}/AIA/{channel}A'
    data = sorted(glob.glob(f'{data_path}/aia_lev1_{channel}a_*.fits'))
    
    start_filename = f"aia_lev1_{channel}a_{dt_dict['start_year']}_{dt_dict['start_month']}_{dt_dict['start_day']}t{dt_dict['start_hour']}_{dt_dict['start_minute']}"
    end_filename   = f"aia_lev1_{channel}a_{dt_dict['end_year']}_{dt_dict['end_month']}_{dt_dict['end_day']}t{dt_dict['end_hour']}_{dt_dict['end_minute']}"
    
    first_file_to_find = glob.glob(f'{data_path}/{start_filename}*.fits')
    last_file_to_find  = glob.glob(f'{data_path}/{end_filename}*.fits')
    
    idx1 = data.index(first_file_to_find[0])
    idx2 = data.index(last_file_to_find[0])
    
    chosen_files = data[idx1:idx2]
    
    map_objects = []
    for i, file in enumerate(chosen_files):
        # load the file as a sunpy map
        m = sunpy.map.Map(file)
        print(f'AIA {channel}A image {i} is loaded')
        if promote:
            # update the metadata of the map to the most recent pointing
            m_updated = update_pointing(m)
            # scale the image to the 0.6"/pix
            # and derotate the image such that the y-axis is aligned with solar North
            m_registered = register(m_updated)
            # exposure time normalization
            m_normalized = m_registered / m_registered.exposure_time
            map_objects.append(m_normalized)
        else:
            map_objects.append(m)
    return map_objects



def save_processed_aia(data=maps_objects, channel=193):
    for i, processed_map in enumerate(data):
        text_string = processed_map.meta['date-obs']
        # make translation table and use it
        translation_table = str.maketrans('-:.', '___')
        result = text_string.translate(translation_table)
        output_filename = f'aia_{channel}a_{result}_lev15'
        file_path = f'/home/mnedal/data/AIA/{channel}A/lv15/{output_filename}.fits'
        if not os.path.exists(file_path):
            processed_map.save(file_path, filetype='auto')
            print(f'Image {i} is exported')
        else:
            print(f'Image {i} exists already')

## Start loading files and save the processed level 1.5 AIA data for easy retrieval later

In [ ]:
aia_131_map_objects = load_aia(start='2024-05-14T17:00:00', end='2024-05-14T19:01:00', promote=True, channel=131)
os.makedirs(f'/home/mnedal/data/AIA/131A/lv15', exist_ok=True)
save_processed_aia(data=maps_objects, channel=131)

In [ ]:
aia_171_map_objects = load_aia(start='2024-05-14T17:00:00', end='2024-05-14T19:01:00', promote=True, channel=171)
os.makedirs(f'/home/mnedal/data/AIA/171A/lv15', exist_ok=True)
save_processed_aia(data=maps_objects, channel=171)

In [ ]:
aia_193_map_objects = load_aia(start='2024-05-14T17:00:00', end='2024-05-14T19:01:00', promote=True, channel=193)
os.makedirs(f'/home/mnedal/data/AIA/193A/lv15', exist_ok=True)
save_processed_aia(data=maps_objects, channel=193)